# Teil 1, Demo 1: Cholera-Daten von John Snow

Ziel dieser Demo ist eine Reproduktion der klassischen Analyse von Cholera-Daten durch John Snow in 1854. Der Datensatz ist frei verfügbar und wird hier schrittweise mit R-Funktionen geladen, aufbereitet und visualisiert.

## Bibliotheken & Konfiguration

In diesem Abschnitt werden benötigte Programmpakete geladen und Konfigurationsvariablen z.B. für die Datenquellen gesetzt.

Hinweis: auf Google Colab kann das Laden der Pakete vor allem beim ersten Aufruf einige Minuten dauern. Bitte führen Sie diesen Block dann nicht erneut aus, sondern warten die Ausführung ab.

In [ ]:
packages <- c("leaflet", "readr", "tidyr", "ggplot2")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-Datenanalyse-Visualisierung/master/Rohdaten/john-snow/"

## Zeitverlauf Infektionen & Todesfälle laden

In diesem Abschnitt laden wir die Daten zum Verlauf der Finektionen ("attacks") und Todesfälle ("deaths") der Cholera-Epidemie von 1854.

In [ ]:
snow.dates.raw <- read_delim(paste(base_url, "snow.dates.csv", sep=""),
                           col_names = c("index", "date", "attacks", "deaths"),
                           col_types = cols("i", col_date(format = "%Y-%m-%d"), "i", "i"),
                           skip = 1, delim = ",")
head(snow.dates.raw)

## Zeitverlauf plotten

In diesem Abschnitt visualisieren wir die Infektionen und Todesfälle über ein Flächendiagramm ("area chart"). Die Infektionen werden dabei rot und die Todesfälle blau dargestellt, wobei die Flächen leicht transparent dargestellt werden, so dass auch überlappende Anteile beurteilt werden können.

In [ ]:
snow.dates.long <- snow.dates.raw %>% pivot_longer(!c(index, date), names_to="variable", values_to="n")
options(repr.plot.width = 10, repr.plot.height = 2.5)
ggplot(snow.dates.long) +
    geom_area(aes(x=date, y=n, fill=variable), alpha=0.5, position="identity")

## Geovisualisierung

Im folgenden stellen wir die Daten der Todesfälle als Geovisualisierung auf einer Karte (des heutigen) Londons dar.

🛑 Wichtig: dieser Abschnitt kann leider auf Google Colab nicht dargestellt werden, da das genutzte `Leaflet`-Package in Colab nur unter Python, nicht jedoch mit R funktioniert. Für diesen Teil der Demo müssen Sie deshalb entweder Binder oder eine lokale Installation von JupyterLab nutzen.

### Koordinaten der Todesfälle laden

In [ ]:
snow.deaths.raw <- read_csv(paste(base_url, "cholera_deaths.csv", sep=""),
                            col_names=c("index", "deaths", "lon", "lat"),
                            col_types = cols("i","i","d","d"),
                            skip=1)
head(snow.deaths.raw)

### Todesfälle auf der Karte anzeigen

Die Todesfälle werden als halbtransparente rote Kreise dargestellt, wobei der Radius der Kreise durch die Anzahl der Todesfälle an der jeweiligen Adresse bestimmt wird.

In [ ]:
if ("COLAB_GPU" %in% names(Sys.getenv())) {
  print("Diese Karte kann unter Google Colab leider nicht angezeigt werden.")
} else {
  leafletmap <- leaflet() %>%
      setView(-0.136722, 51.513578, zoom=16) %>%
      addProviderTiles(providers$OpenStreetMap.DE) %>%
      addCircleMarkers(
          data = snow.deaths.raw, ~lon, ~lat,
          radius = ~deaths * 2,
          color = "red",
          stroke = FALSE,
          fillOpacity = 0.5
      )
  leafletmap
}

### Koordinaten der Wasserpumpen laden

John Snow vermutete, dass die Cholera durch verseuchtes Wasser übertragen wurde. Er zeichnete deshalb die Orte der Wasserpumpten auf seiner Karte ein. Im folgenden ergänzen wir die Pumpen als blaue Marker auf der Karte.

In [ ]:
snow.pumps.raw <- read_csv(paste(base_url, "johnsnow_pumps.csv", sep=""),
                           col_names=c("index", "lon", "lat"),
                           col_types = cols("i","d","d"),
                           skip=1)
head(snow.pumps.raw)

### Wasserpumpen auf der Karte anzeigen

In [ ]:
if ("COLAB_GPU" %in% names(Sys.getenv())) {
  print("Diese Karte kann unter Google Colab leider nicht angezeigt werden.")
} else {

  # Wasserpumpen als Marker auf der Karte ergänzen
  leafletmap <- leafletmap %>%
    addMarkers(data = snow.pumps.raw, ~lon, ~lat)

  # Karte ausgeben
  leafletmap
}